In [12]:
#!pip install weaviate-client==3.21.0

  Using cached weaviate_client-3.21.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached validators-0.21.0-py3-none-any.whl.metadata (2.6 kB)
Using cached weaviate_client-3.21.0-py3-none-any.whl (99 kB)
Using cached validators-0.21.0-py3-none-any.whl (27 kB)
  Attempting uninstall: validators
    Found existing installation: validators 0.22.0
    Uninstalling validators-0.22.0:
      Successfully uninstalled validators-0.22.0
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.3b1
    Uninstalling weaviate-client-4.3b1:
      Successfully uninstalled weaviate-client-4.3b1


In [26]:
import weaviate
import json
import os

client = weaviate.Client(
   url = "http://weaviate:8080"  # Replace with your endpoint

 )

nodes_status = client.cluster.get_nodes_status()
print(nodes_status)


[{'batchStats': {'queueLength': 0, 'ratePerSecond': 6}, 'gitHash': 'a843fb4', 'name': 'node1', 'shards': [{'class': 'TEXT', 'name': 'UoJHExsb9ESr', 'objectCount': 121142}, {'class': 'Question2', 'name': '23rDJTelRNmo', 'objectCount': 10}, {'class': 'Providers', 'name': 'GziX2tL1XoQw', 'objectCount': 0}, {'class': 'PROVIDER', 'name': '3FD4qB7mFgeA', 'objectCount': 8373}], 'stats': {'objectCount': 129525, 'shardCount': 4}, 'status': 'HEALTHY', 'version': '1.21.2'}]


In [15]:
if client.schema.exists("Providers"):
    client.schema.delete_class("Providers")
class_obj = {
    "class": "Providers",
    "vectorizer": "text2vec-transformers"
}

client.schema.create_class(class_obj)
print("Created schema, using text2vec")

Created schema, using text2vec


In [24]:
import os
from snowflake.snowpark import Session

def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': os.getenv("SNOWFLAKE_ROLE"),
    'warehouse': 'COMPUTE_WH',
    'database': os.getenv("SNOWFLAKE_DATABASE"),
    'schema': os.getenv("SNOWFLAKE_SCHEMA")
}

session = Session.builder.configs(connection_params).create()
session = Session.builder.configs(connection_params).create()
session.get_current_warehouse(), session.get_current_role(), session.get_current_database(), session.get_current_schema()
providers=session.table("WEAVIATE_DB.PUBLIC.PROVIDERS")
providers.show()

-----------------------------------------------------------------------------------------------
|"TEXT"                                              |"SPECIALTY"              |"NPI"         |
-----------------------------------------------------------------------------------------------
|The Individual with the NPI of 1427085166 is pa...  |"Social Worker"          |"1427085166"  |
|The Individual with the NPI of 1205064219 is pa...  |"Family Medicine"        |"1205064219"  |
|The Individual with the NPI of 1710345020 is pa...  |"Physician Assistant"    |"1710345020"  |
|The Individual with the NPI of 1144977455 is pa...  |"Nurse Practitioner"     |"1144977455"  |
|The Individual with the NPI of 1053068361 is pa...  |"Psychologist Clinical"  |"1053068361"  |
|The Individual with the NPI of 1114455714 is pa...  |"Physical Therapist"     |"1114455714"  |
|The Individual with the NPI of 1164901492 is pa...  |"Physician Assistant"    |"1164901492"  |
|The Individual with the NPI of 17803310

In [ ]:
print("importing data...")

import pandas as pd
 

# Settings for displaying the import progress
counter = 0
interval = 20  # print progress every this many records; should be bigger than the batch_size

def add_object(obj) -> None:
    global counter
    properties = {
        'TEXT': obj['TEXT'],
        'SPECIALTY': obj['SPECIALTY'],
        'NPI': obj['NPI']
    }

    client.batch.configure(batch_size=100)  # Configure batch
    with client.batch as batch:
        # Add the object to the batch
        batch.add_data_object(
            data_object=properties,
            class_name='PROVIDER',
            # If you Bring Your Own Vectors, add the `vector` parameter here
            # vector=obj.vector
        )

        # Calculate and display progress
        counter += 1
        if counter % interval == 0:
            print(f'Imported {counter} providers...')


# Looping over the DataFrame and constructing dictionaries for each row
for index, row in providers.to_pandas().iterrows():
    # Constructing a dictionary for the current row
    row_dict = row.to_dict()
    
    # The keys in row_dict are the column names of the DataFrame, and the values are the corresponding data in each row
    
    # Here you can use the row_dict to upload to the Weaviate database
    print(row_dict)  # This line is just to demonstrate what the row_dict looks like, you can remove it in actual use
    add_object(row)

 
print(f'Finished importing {counter} providers.')


importing data...
{'TEXT': 'The Individual with the NPI of 1427085166 is part of the location Camden with a specialty of Social Worker is Thompson, Angela', 'SPECIALTY': '"Social Worker"', 'NPI': '"1427085166"'}
{'TEXT': 'The Individual with the NPI of 1205064219 is part of the location Gadsden with a specialty of Family Medicine is Uy, Karl Peter', 'SPECIALTY': '"Family Medicine"', 'NPI': '"1205064219"'}
{'TEXT': 'The Individual with the NPI of 1710345020 is part of the location Falls Church with a specialty of Physician Assistant is Ufomata, Kimberly', 'SPECIALTY': '"Physician Assistant"', 'NPI': '"1710345020"'}
{'TEXT': 'The Individual with the NPI of 1144977455 is part of the location Syracuse with a specialty of Nurse Practitioner is Gaube, Kimberly', 'SPECIALTY': '"Nurse Practitioner"', 'NPI': '"1144977455"'}
{'TEXT': 'The Individual with the NPI of 1053068361 is part of the location Winnetka with a specialty of Psychologist Clinical is Amezcua, Lorena', 'SPECIALTY': '"Psychologi

In [ ]:
#Semantic Search Example

In [32]:
nearText = {
        "concepts": ["cancer"],
    }

properties = [
        "tEXT",
        "_additional {distance}"
    ]

response = (
        client.query
        .get("PROVIDER", properties)
        .with_near_text(nearText)
        .with_limit(3)
        .do()
    )



print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "PROVIDER": [
                {
                    "_additional": {
                        "distance": 0.696097
                    },
                    "tEXT": "The Individual with the NPI of 1518072701 is part of the location Cambridge with a specialty of Radiation Oncology is Jueng, Carl"
                },
                {
                    "_additional": {
                        "distance": 0.7155723
                    },
                    "tEXT": "The Individual with the NPI of 1942535786 is part of the location Fairfax with a specialty of Surgical Oncology is Lee, David"
                },
                {
                    "_additional": {
                        "distance": 0.7181144
                    },
                    "tEXT": "The Individual with the NPI of 1700233228 is part of the location Saginaw with a specialty of Radiation Oncology is Damico, Nicholas"
                }
            ]
        }
    }
}


In [ ]:
#Hybrid Search Example

In [35]:
response = (
    client.query
    .get("PROVIDER", ["tEXT", "nPI"])
    .with_hybrid(
        query="cancer",
    )
    .with_additional(["score", "explainScore"])
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "PROVIDER": [
        {
          "_additional": {
            "explainScore": "(vector) [0.3920454 0.5924074 0.30148762 0.04340668 -0.4266341 0.27028736 0.6954215 0.30716822 -0.10130334 -0.1874528]...  \n(hybrid) Document 10db13e3-282d-43d3-b9eb-fdd49bd34529 contributed 0.012295081967213115 to the score",
            "score": "0.012295082"
          },
          "nPI": "\"1518072701\"",
          "tEXT": "The Individual with the NPI of 1518072701 is part of the location Cambridge with a specialty of Radiation Oncology is Jueng, Carl"
        },
        {
          "_additional": {
            "explainScore": "(vector) [0.3920454 0.5924074 0.30148762 0.04340668 -0.4266341 0.27028736 0.6954215 0.30716822 -0.10130334 -0.1874528]...  \n(hybrid) Document cfdf74c8-bec3-4801-8898-5000141b5079 contributed 0.012096774193548387 to the score",
            "score": "0.012096774"
          },
          "nPI": "\"1942535786\"",
          "tEXT": "The Individual with